In [4]:
from myutils import *
import rasterio
from rasterio.transform import from_origin
import pickle

from datetime import datetime
from datetime import date

import requests
import asyncio
import aiohttp
import time

import os
import gc

from shapely import Point
import rasterio
from rasterio.merge import merge

In [5]:
# Carga el polígono de la ciudad de méxico, aplica un buffer de 100 mts y lo proyecta a WGS84
cdmx = gpd.read_file(r'../GeoData/Polygons.gpkg', layer='CDMX', encoding='utf-8')
polygon = cdmx.geometry[0].buffer(100)
cdmx['geometry'] = polygon
cdmx.to_crs(4326, inplace=True)
polygon = cdmx.geometry[0]

# Obtiene el bounding box del polígono
(lon1 , lat1, lon2, lat2) = polygon.bounds

In [6]:
# Cálculo de la resolución de las teselas

# Nivel de zoom
zoom = 16

# Primera y última tesela
lower = lat_lon_to_tile_zxy(lat1, lon1, zoom)
upper = lat_lon_to_tile_zxy(lat2, lon2, zoom)

# Tamaño de la imagen a descargar (512x512 o 256x256)
imsize = int(512)
print(lower,upper)

# Número de teselas por lado
tiles_x = (upper[1] - lower[1] + 1)
tiles_y = (lower[2] - upper[2] + 1)
print(f'Teselas totales: {tiles_x}x{tiles_y} = {tiles_x*tiles_y}')

# Coordenadas en lat, lon de las esquinas de la imagen, en el sistema de referencia WGS84
inf_izq = tile_zxy_to_lat_lon(zoom, lower[1], lower[2] + 1)
sup_der = tile_zxy_to_lat_lon(zoom, upper[1] + 1, upper[2])
sup_izq = (sup_der[0], inf_izq[1])
inf_der = (inf_izq[0], sup_der[1])

# Cálculo de la resolución de las teselas
circunferencia = 40075017
resolucion_tiles = circunferencia/(2**zoom) # metros
resolucion_pixeles = resolucion_tiles/imsize # metros/pixel

print('Coordenadas reales:', inf_izq, sup_der)
print('resolucion en pixeles', resolucion_pixeles, 'metros/pixel')

[16, 14678, 29234] [16, 14756, 29129]
Teselas totales: 79x106 = 8374
Coordenadas reales: [19.04654131204215, -99.371337890625] [19.59601924031249, -98.9373779296875]
resolucion en pixeles 1.1943285763263702 metros/pixel


In [7]:
# Genera la geometría de las teselas
tiles_bbox = {'x':[], 'y':[], 'lat':[], 'lon':[], 'geometry':[], 'intersect':[]}

# Identifica las teselas que intersectan con el polígono de la ciudad de méxico
# para descargar únicamente las teselas que contienen información de la ciudad
queries = []

# Recorre los índices de las teselas
for i in range(lower[1], upper[1]+1):
    for j in range(upper[2], lower[2]+1):
 
        y1,x1,y2,x2 = tile_zxy_to_lat_lon_bbox(zoom,i,j)
        
        point1 = (x1,y1)
        point2 = (x1,y2)
        point3 = (x2,y2)
        point4 = (x2,y1)
        
        # Genera la geometría de la tesela
        tile = sp.Polygon([point1,point2,point3,point4])
        
        # Almacena la geometría de la tesela
        tiles_bbox['lon'].append(i)
        tiles_bbox['lat'].append(j)
        tiles_bbox['x'].append(x1)
        tiles_bbox['y'].append(y1)
        tiles_bbox['geometry'].append(tile)
        
        # Si la tesela intersecta con el polígono de la ciudad de méxico
        if tile.intersects(polygon):
            
            
            tiles_bbox['intersect'].append(1)
            
            # Marca la tesela para descargar
            queries.append({'x': i, 'y':j, 'query': 1, 'sup_izq': (y1,x1), 'inf_der': (y2,x2)})
        
        else:
            # Marca la tesela para no descargar
            queries.append({'x': i, 'y':j, 'query': 0, 'sup_izq': [], 'inf_der': []})
            tiles_bbox['intersect'].append(0)
        
bbox = sp.Polygon([(inf_izq[1], inf_izq[0]), (sup_der[1], inf_izq[0]), (sup_der[1], sup_der[0]), (inf_izq[1], sup_der[0])])
tiles_bbox['x'].insert(0, 0)
tiles_bbox['y'].insert(0, 0)
tiles_bbox['lat'].insert(0, 0)
tiles_bbox['lon'].insert(0, 0)
tiles_bbox['intersect'].insert(0, -1)
tiles_bbox['geometry'].insert(0, bbox)

# Convierte las teselas a un GeoDataFrame
tiles_bbox = gpd.GeoDataFrame(tiles_bbox, crs=4326)

# Número de teselas a descargar
tiles_used = sum([ 1 if i['query'] == 1 else 0 for i in queries])

In [ ]:
# tiles_bbox.explore(column='intersect', cmap='tab20', tooltip=True)

In [ ]:
# tiles_bbox.to_file('../GeoData/Traffic/Mini/_tiles_bbox.gpkg', layer='tiles', driver="GPKG", encoding="utf-8")
tiles_bbox.to_file('../GeoData/Traffic/Ej/tiles_bbox.gpkg', layer='tiles', driver="GPKG", encoding="utf-8")
del(tiles_bbox)

In [ ]:
# Función para generar la url de descarga de las teselas
def get_url(x,y,zoom, imsize, apikey):
    url = f'https://api.tomtom.com/traffic/map/4/tile/flow/relative0/{zoom}/{x}/{y}.png?tileSize={imsize}&key=' + apikey
    return url

## Consulta asíncrona de teselas

In [ ]:
# Obtiene la fecha y hora actual
fecha = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Api key de TomTom
try:
    # Api key de TomTom
    with open('../GeoData/tomtom_apikey.txt', 'r') as f:
        apiKey = f.read()
        f.close()
except:
    print('No se pudo leer el archivo con la API key de TomTom')
    print('se debe crear un archivo "tomtom_apikey.txt" con la API key en: ', '../GeoData/')

# Lista para almacenar las imágenes
tiles = dict([(i, dict([ (j,'') for j in range(upper[2], lower[2]+1)] )) for i in range(lower[1], upper[1]+1)])
attempts = 5
failed = []

async def get(query, session):
    try:
        
        x = query['x']
        y = query['y']
        
        if query['query'] == 1:
            url_query = get_url(query['x'], query['y'], zoom, imsize, apiKey)
            
            async with session.get(url=url_query) as response:
                
                if response.status != 200:
                    for _ in range(attempts):
                        await asyncio.sleep(1)  # Esperar 1 segundo antes de volver a intentar
                        
                        # Hacer un nuevo intento con la misma URL
                        async with session.get(url=url_query) as new_response:
                            if new_response.status == 200:
                                resp = await new_response.read()
                                image = np.array(Image.open(BytesIO(resp)))
                                tiles[x][y] = image
                                print(f'Imagen obtenida: {x},{y}')
                                break  # Si se obtiene una respuesta exitosa, salir del bucle
                            
                    else:
                        print(f'Error al obtener imagen para {x},{y}. Se agotaron los intentos.')
                        failed.append(query)
                else:
                    resp = await response.read()
                    image = np.array(Image.open(BytesIO(resp)))
                    tiles[x][y] = image
                    print(f'Imagen obtenida: {x},{y}')
        else:
            tiles[x][y] = 0
            
            
    except Exception as e:
        print("Unable to get url {} due to {}.".format(query, e.__class__))
        

async def main(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[get(url, session) for url in urls])
    print(f'Finalizado, faltaron {len(failed)} imágenes')


start = time.time()
result = await main(queries)
end = time.time()

## Consulta secuencial de Tiles

In [ ]:
# %%time
# # Api key de TomTom
# try:
#     # Api key de TomTom
#     with open(path + 'tomtom_apikey.txt', 'r') as f:
#         apiKey = f.read()
#         f.close()
# except:
#     print('No se pudo leer el archivo con la API key de TomTom')
#     print('se debe crear un archivo "tomtom_apikey.txt" con la API key en: ', path_source)
# images = []

# for url in urls:
    
#     if url['query'] == 1:
        
#         response = requests.get(url['url'])
#         if response.status_code == 200:
#             image = np.array(Image.open(BytesIO(response.content)))
#             images.append([url['x'], url['y'], image])
#             x = url['x']
#             y = url['y']
#             print(f'Imagen obtenida: {x},{y}, {len(images)}/{tiles_x*tiles_y}')
#         else: 
#             print("Fallo en la descarga de la imagen")
    
#     else:
#         images.append([url['x'], url['y'], np.zeros((imsize,imsize,4))])

## Conversión de tesela a monobanda

In [ ]:
def rgba_to_mono(tile):
    
    # Máscara para identificar los pixeles con información
    mask = tile[:,:,3] > 250

    # Obtener los canales RGB y aplicar la máscara
    tile2 = tile[:,:,0:3] * np.expand_dims(mask, axis = 2)
    
    # Posteriza la imágen a 4 valores por canal
    tile2 = tile2/(255/4)
    tile2 = tile2.round() * (255/4)
    
    # Aplica la fórmula 5*R + G + 10*B
    res = 5*tile2[...,1] + tile2[...,0] + 10*tile2[...,2]

    # Define los colores de la imagen
    # colores = [0, 206, 1594, 1913, 1721, 1211, 4335]
    colores = [0, 1657, 1848,1530, 212, 1148,  4080  ]
    
    # Define las tolerancias para cada color
    tols = np.array([0, 10, 10, 1, 50, 10, 10])

    # Calcula la diferencia entre la imágen y los colores
    dif  = np.abs(res - np.expand_dims(colores,axis=[1,2]))
    
    # Obtiene el índice del color más cercano
    ind_min = np.argmin(dif,axis=0).astype(np.intp)
    
    # Asigna el color más cercano sólo si la diferencia es menor a la tolerancia
    ima =  ind_min * (dif.min(axis=0) <= np.expand_dims(tols,axis=[1,2]).take(ind_min))
    
    return ima.astype(np.uint8)

## Guardar el objeto para consulta por tesela

In [ ]:
path = '../GeoData/Traffic/Ej/'
name = f'traffic_flow_{zoom}_{fecha}'

In [ ]:
# for q in queries:
#     if q['query'] == 1:
        
#         tiles[q['x']][q['y']] = {'array' : tiles[q['x']][q['y']], 
#                                  'sup_izq': q['sup_izq'], 
#                                  'inf_der': q['inf_der']}

# ##Para guardar el diccionario con las teselas como RGBA
# with open(path + name + '_rgba.pkl', 'wb') as file:
#     pickle.dump(tiles, file)
#     print(f'Teselas guardadas como "{path + name}_rgba.pkl"')

In [ ]:
# Aplica la función para convertir las imágenes a monocromáticas
for q in queries:
    if q['query'] == 1:
        
        tiles[q['x']][q['y']] = {'array' : rgba_to_mono(tiles[q['x']][q['y']]), 
                                 'sup_izq': q['sup_izq'], 
                                 'inf_der': q['inf_der']}

# Para guardar el diccionario con las teselas como imágenes monocromáticas
# (sobre escribe la variable tiles, por lo que se recomienda guardar primero las teselas RGBA)
with open(path + name + '.pkl', 'wb') as file:
    pickle.dump(tiles, file)
    print(f'Teselas guardadas como "{path + name}.pkl"')

## Creación de una imágen completa

In [ ]:
# Para trabajar con las teselas guardadas previamiente en otra sesión
# path = '../GeoData/Traffic/Ej/'
# name = 'traffic_flow_16_2023-11-14_13-45-00_rgba'
# tiles = pickle.load(open(path + name + '.pkl', 'rb'))

In [ ]:
def save_geotif(tiles,upper,lower,imsize,zoom,path,name,rgba=False):

    ### ----- Creación de filas como raster ----- ###
    
    # Ruta completa de la carpeta "temp" para almacenar los rasters individuales
    # de cada fila temporalmente
    
    ruta_temp = os.path.join(path, "temp")

    # Verificar si la carpeta "temp" existe
    if not os.path.exists(ruta_temp):
        # Si no existe, crearla
        os.makedirs(ruta_temp)
    
    # Lista para almacenar los nombres de los rasters individuales
    filas_temp = []

    # Recorre los índices de las filas
    for j in range(upper[2], lower[2]+1):
        
        # Lista para almacenar las teselas de la fila
        fila = []
        
        # Recorre los índices de las columnas
        for i in range(lower[1], upper[1]+1):
            
            # Si la tesela existe, se agrega a la fila
            if tiles[i][j]:
                # tiles[i][j][0][0] = np.array([0,0,0,255])
                # tiles[i][j][-1][-1] = np.array([255,0,0,255])
                fila.append(tiles[i][j]['array'].astype(np.uint8))
            
            # Si no existe, se agrega una tesela vacía
            else:
                if rgba:
                    fila.append(np.zeros((imsize,imsize,4), dtype=np.uint8))
                
                else:
                    fila.append(np.zeros((imsize,imsize), dtype=np.uint8))
                
        
        fila = np.hstack(fila)
        
        # Calcula las coordenadas en lat, lon de las esquinas de la tesela    
        y1, x1, y2, x2 = tile_zxy_to_lat_lon_bbox(zoom,lower[1],j)
        x1, x2 = sup_izq[1], sup_der[1]
        
        # def from_bounds(west, south, east, north, width, height):
        transformacion = rasterio.transform.from_bounds(x1,y2,x2,y1,imsize*tiles_x,imsize)
        
        if rgba:
            banda = 4
        else:
            banda = 1
        
        row_name = f'{path}temp/row_{j}.tif'
        new_dataset = rasterio.open(row_name, 'w', driver='GTiff',
                                    height = fila.shape[0], width = fila.shape[1],
                                    count=banda, dtype=str(fila.dtype),
                                    crs='EPSG:4326',
                                    transform=transformacion)

        if rgba:
            new_dataset.write(np.rollaxis(fila, axis=2) )
        else:
            # new_dataset.write(np.rollaxis(np.expand_dims(fila,axis=-1), axis=2))
            new_dataset.write(fila,1)
            
        new_dataset.close()
        del(new_dataset)
        
        filas_temp.append(row_name)
    
    
    
    ### ----- Combinación de filas en una sola imagen ----- ###

    # Cargar los rasters individuales
    rasters = []

    for ruta_raster in filas_temp:
        rasters.append(rasterio.open(ruta_raster) )

    # Combinar los rasters en una sola imagen
    merged, out_transform = merge(rasters)

    # Actualizar la información de georreferenciación si es necesario
    out_profile = rasters[0].profile
    out_profile.update(transform=out_transform, width=merged.shape[2], height=merged.shape[1])

    # Guardar la imagen combinada
    with rasterio.open(path+name+'.tif', 'w', **out_profile) as dst:
        dst.write(merged)
        written = True

    if written:
        print(f'Imagen guardada como "{path + name}.tif"')
        # Eliminar los rasters
        del(merged)
        del(rasters)
        for ruta_raster in filas_temp:
            os.remove(ruta_raster)
    
    return

In [ ]:
# Guarda la imágene como un GeoTiff
save_geotif(tiles,upper,lower,imsize,zoom,path,name,rgba=False)

In [ ]:
# Carga el grafo de la red de transporte de la Ciudad de México
g = ox.load_graphml(filepath="../GeoData/graph_transport.graphml")

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(g, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)
# gdf_edges['geometry'] = gdf_edges['geometry'].to_crs(4326)

In [ ]:
# Función para obtener el punto medio de una línea
def middle_point(geom):
    
    # Obtiene las coordenadas de los puntos que forman la línea
    lats, lons = geom.xy
    lats = lats.tolist()
    lons = lons.tolist()
    
    
    
    # Si la línea tiene dos puntos, 'a' es el punto medio y 'b' es el punto final
    if len(lats) == 2:
        mid_point = [sum(lats)/2, sum(lons)/2]
        a = np.array(mid_point)
        b = np.array([lats[1], lons[1]])
    
    # Si la línea tiene más de dos puntos, 'a' es el punto medio de los dos puntos centrales
    # y b es el punto central posterior a 'a'
    else:
        
        middle = int(len(lats)/2)
        a = np.array([sum(lats[middle-1:middle+1]), sum(lons[middle-1:middle+1])])/2
        b = np.array([lats[middle], lons[middle]])
    
    # Calcula el vector que va de 'a' a 'b'
    vec_ab = b - a
    
    # Calcula el vector perpendicular a 'ab'
    perpendicular = np.array([vec_ab[1], -vec_ab[0]])
    
    # Normaliza el vector perpendicular y lo traslada al punto 'a', lo desplaza 3 metros
    perpendicular = 3*perpendicular/np.linalg.norm(perpendicular)
    perpendicular += a
    
    # Regresa el punto extremo del vector perpendicular
    return Point(perpendicular[0], perpendicular[1])


In [ ]:
# Función para muestrar una imagen geo-referenciada a partir de un punto
def sample_raster(point, zoom, tiles, imsize, tiles2 = ''):
    # Obtiene las coordenadas del punto
    lat = point.y
    lon = point.x
    
    # Obtiene las coordenadas de la tesela correspondiente
    z, x,y = lat_lon_to_tile_zxy(lat, lon, zoom)
    lat1, lon1 = tiles[x][y]['sup_izq']
    lat2, lon2 = tiles[x][y]['inf_der']
    
    # Normaliza las coordenadas, considerando las esquinas van de 0 a imsize
    centro_x = math.floor(imsize * (lon - lon1)/(lon2 - lon1))
    centro_y = math.floor(imsize * (lat - lat1)/(lat2 - lat1))
    
    # Pixel correspondiente al centroide
    c = [centro_y, centro_x]
    
    # radio de la ventana
    r = 5
    window = tiles[x][y]['array'][max(c[0] - r, 0): min(imsize - 1, c[0] + r),
                                  max(c[1] - r, 0): min(imsize - 1, c[1] + r)]
    
    # Cuenta la frecuencia de cada valor en la ventana
    vals, counts = np.unique(window, return_counts=True)
    
    # Si sólo hay un valor, asigna el valor más común
    if len(vals) == 1:
        most_common = vals[0]
        
        # Si el valor más común es 0, asigna 1 (libre)
        if most_common == 0:
            most_common = 1

    # Si hay múltiples valores:
    else:
        
        # Encuentra el valor más común en la ventana, sin considerar el 0
        most_common = vals[np.argmax(counts[1::]) + 1]
    
    # Si se adjunta una segunda matriz, asigna el valor más común a la ventana correspondiente
    if tiles2:
        tiles2[x][y]['array'][max(c[0] - r, 0): min(imsize - 1, c[0] + r),
                             max(c[1] - r, 0): min(imsize - 1, c[1] + r)] = most_common
    
    return most_common
    
    

In [ ]:
# Obtiene los puntos medios de las aristas para sacar la muestra
gdf_edges['middle_point'] = gdf_edges.apply(lambda row : middle_point(row['geometry']), axis = 1)

# Crea un objeto GeoSeries con los puntos medios de las aristas
gdf_edges_mid = gdf_edges['middle_point'].copy()
gdf_edges_mid.rename('geometry', inplace=True)

# Por defecto estan en el sistema de referencia del grafo, por lo que se convierten a WGS84
# para poder obtener las teselas correspondientes
gdf_edges_mid.crs = gdf_edges.crs
gdf_edges_mid = gdf_edges_mid.to_crs(4326)

In [ ]:
# # Crea una copia de las teselas para asignarles el valor de la muestra
tiles2 = copy(tiles)
traffic_data = gdf_edges_mid.apply(lambda row : sample_raster(row, zoom, tiles, imsize,tiles2=tiles2))
traffic_data = pd.DataFrame(traffic_data)

In [ ]:
# Guarda las teselas con la ventana de muestreo marcada
save_geotif(tiles2,upper,lower,imsize,zoom,path,name+"_sampled",rgba=False)

In [ ]:
traffic_data.to_pickle(f'{path}{name}_traffic_data.tar')

In [ ]:
traffic_data = pd.read_pickle(f'{path}{name}_traffic_data.tar') 
gdf_edges = gdf_edges.assign(trafico= traffic_data)

In [ ]:
gdf_edges.head()
gdf_edges[['trafico']].head(10)

In [ ]:
new_g = ox.graph_from_gdfs(gdf_nodes, gdf_edges)

In [ ]:
save_graph_geopackage(new_g, filepath=f'{path}{name}_graph_traffic.gpkg', encoding='utf-8')


In [ ]:
# Para observar en qué punto se hace el muestreo
gdf_edges_mid.to_file(path + 'middle_point.gpkg', layer='edges', driver="GPKG", encoding="utf-8")